In [1]:
# Import necessary PyTorch libraries
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms


# Additional libraries for visualization and utilities
import matplotlib.pyplot as plt
import numpy as np
from unet_decoder import UNetDecoder

In [2]:
def get_device():
    """Selects the best available device for PyTorch computations.

    Returns:
        torch.device: The selected device.
    """

    if torch.cuda.is_available():
        return torch.device('cuda')
    elif torch.backends.mps.is_available():
        return torch.device('mps')
    else:
        return torch.device('cpu')

device = get_device()
print(f"using device: {device}")

using device: cuda


In [3]:
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import Compose, Normalize, ToTensor,Resize

from torch.utils.data import DataLoader, random_split

# Define the transformation with resizing
transform = transforms.Compose([
    Resize((28,28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize with MNIST mean and std
])

# Load the Omniglot dataset
dataset = datasets.Omniglot(root='./data', download=True, transform=transform)

# Print the total number of images in the dataset
print(f"Total number of images in the dataset: {len(dataset)}")

# Splitting dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Print the number of images in the train and validation sets
print(f"Number of images in the training set: {len(train_dataset)}")
print(f"Number of images in the validation set: {len(val_dataset)}")

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

Files already downloaded and verified
Total number of images in the dataset: 19280
Number of images in the training set: 15424
Number of images in the validation set: 3856


In [4]:
class ColdDiffusionModel(nn.Module):
    def __init__(self, input_shape, T=1000):
        super(ColdDiffusionModel, self).__init__()
        self.input_shape = input_shape
        self.T = T
        self.decoder = UNetDecoder(n_channels=input_shape[0])

        # Define the noise schedule
        self.alpha = self.create_noise_schedule(T)

    def create_noise_schedule(self, T):
        beta_start = 0.0001
        beta_end = 0.02
        betas = torch.linspace(beta_start, beta_end, T)
        alphas = 1 - betas
        alphas_cumprod = torch.cumprod(alphas, dim=0)
        return alphas_cumprod

    def forward(self, x):

        # Calculate the gaussian noise tensor
        batch_size = x.shape[0]
        epsilon = torch.randn(batch_size, 1, 28, 28).to(device)
        
        torch.cuda.empty_cache()

        #sample a timestep t
        t = np.random.randint(0, self.T)
        # Retrieve noise scheduler alpha_T
        alpha_t = self.alpha[t]

        # Calculate square root alphas
        sqrt_alpha_t = torch.sqrt(alpha_t)
        sqrt_one_minus_alpha_t = torch.sqrt(1 - alpha_t)
        
        # Perform the weighted sum
        x_t = sqrt_alpha_t * x + sqrt_one_minus_alpha_t * epsilon

        torch.cuda.empty_cache()

        #Calculate the timestep tensor
        t = torch.tensor([t] * x_t.size(0), dtype=torch.long).to(x_t.device)

        # Perform the reconstruction process using Algorithm 2
        estimated_x = self.decoder(x_t,t)
        torch.cuda.empty_cache()
        return estimated_x

In [9]:
import os
import torch

def save_checkpoint(epoch, model, optimizer, filename="checkpoint.pth"):
    """Saves the model and optimizer state at the specified path."""
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, filename)
    print(f"Checkpoint saved at epoch {epoch} to {filename}")

def load_checkpoint(model, optimizer, filename="checkpoint.pth", device='cpu'):
    """Loads the model and optimizer state from the specified path."""
    if os.path.isfile(filename):
        checkpoint = torch.load(filename, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        print(f"Checkpoint loaded from {filename}, resuming training from epoch {epoch}")
        return epoch
    else:
        print(f"No checkpoint found at {filename}, starting from scratch.")
        return -1

In [6]:
import time  # Importing time to log the duration
from tqdm import tqdm


def validate(model, val_loader, device):
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    with torch.no_grad():  # Disable gradient computation during validation
        for data, _ in val_loader:
            data = data.to(device)
            estimated_image = model(data)
            reconstruction_loss = nn.functional.mse_loss(data, estimated_image)
            total_val_loss += reconstruction_loss.item()  # Accumulate the validation loss
    
    avg_val_loss = total_val_loss / len(val_loader)  # Calculate average loss
    return avg_val_loss

def train(model, optimizer, train_loader, device, start_epoch, num_epochs):
    model.train()

    for epoch in range(start_epoch+1, num_epochs):
        epoch_loss = 0.0
        epoch_start_time = time.time()  # Time tracking for the epoch

        print(f"Starting epoch {epoch+1}/{num_epochs}")
        for batch_idx, (data, _) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)):
            data = data.to(device)

            # Forward pass
            estimated_image = model(data)
            total_loss = nn.functional.mse_loss(data, estimated_image)

            # Backward pass
            if not torch.isnan(total_loss).any():
                total_loss.backward()
            else:
                print(f"Warning: NaN detected in total_loss at batch {batch_idx+1}, skipping backward pass.")

            optimizer.step()  # Only step the optimizer every `accumulation_steps`
            optimizer.zero_grad()  # Reset gradients only after accumulation

            # Safe-guarding against NaN for epoch_loss
            if not torch.isnan(total_loss).any():
                epoch_loss += total_loss.item()
            else:
                print(f"NaN detected, not adding to epoch_loss at batch {batch_idx+1}")

        # Save the model checkpoint
        save_checkpoint(epoch, model, optimizer, filename="mnist_cold_diffusion.pth")
        
        # Average loss after training for an epoch
        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}] completed in {time.time() - epoch_start_time:.2f} seconds, Avg Loss: {avg_loss}")

        # Validation phase
        avg_val_loss = validate(model, val_loader, device)
        print(f"Epoch [{epoch+1}/{num_epochs}] validation completed, Avg Validation Loss: {avg_val_loss}")

    return model


In [10]:
# Define the input shape
input_shape = (1, 28, 28)

# Create an instance of Gaussian Diffusion model
model = ColdDiffusionModel(input_shape).to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr = 1e-3)


# Load the model training checkpoint
start_epoch = load_checkpoint(model, optimizer, filename="omniglot_cold_l1.pth")
    
print(f"The training ended in epoch number: {start_epoch}")



Checkpoint loaded from omniglot_cold_l1.pth, resuming training from epoch 399
The training ended in epoch number: 399


## Sampling according to Algorithm 1

In [11]:
import torch
import torch.nn.functional as F

# Define the necessary parameters and variables
T = 1000
beta_start = 0.0001
beta_end = 0.02
beta = torch.linspace(beta_start, beta_end, T)
alpha = 1 - beta
alpha = torch.cumprod(alpha, dim=0)
alpha_T = alpha[-1].to(device)  # Move alpha_T to the same device as data and x

model.eval()

total_val_loss = 0  # Initialize total validation loss

with torch.no_grad():
    for batch_idx, (data, _) in enumerate(val_loader):
        data = data.to(device)
        x = torch.randn(data.shape[0], 1, 28, 28).to(device)
        x = torch.sqrt(alpha_T) * data + torch.sqrt(1 - alpha_T) * x
        for s in range(T-1, 0, -1):
            t = torch.tensor([s] * x.size(0), dtype=torch.long).to(device)
            x_hat = model.decoder(x, t)
            z_hat = (1.0 / torch.sqrt(1 - alpha[s])) * (x - torch.sqrt(alpha[s]) * x_hat)
            x = torch.sqrt(alpha[s-1]) * x_hat + torch.sqrt(1 - alpha[s-1]) * z_hat
            
        # Calculate the validation reconstruction loss
        reconstruction_loss = torch.nn.functional.mse_loss(data, x)
        print(f"Reconstruction loss for the batch: {reconstruction_loss.item()}")
        total_val_loss += reconstruction_loss.item()

avg_val_loss = total_val_loss / len(val_loader)

print(f"Average reconstruction loss: {avg_val_loss}")


Reconstruction loss for the batch: 0.22159691154956818
Reconstruction loss for the batch: 0.23741087317466736
Reconstruction loss for the batch: 0.21319256722927094
Reconstruction loss for the batch: 0.2336176484823227
Reconstruction loss for the batch: 0.22548748552799225
Reconstruction loss for the batch: 0.232516810297966
Reconstruction loss for the batch: 0.22869062423706055
Reconstruction loss for the batch: 0.23413525521755219
Reconstruction loss for the batch: 0.22419339418411255
Reconstruction loss for the batch: 0.2183477133512497
Reconstruction loss for the batch: 0.24010556936264038
Reconstruction loss for the batch: 0.2416621744632721
Reconstruction loss for the batch: 0.2374241054058075
Reconstruction loss for the batch: 0.23542693257331848
Reconstruction loss for the batch: 0.22354669868946075
Reconstruction loss for the batch: 0.22343279421329498
Reconstruction loss for the batch: 0.22954368591308594
Reconstruction loss for the batch: 0.23452357947826385
Reconstruction l

## Sampling according to Algorithm 2

In [12]:
import torch
import torch.nn.functional as F

# Define the necessary parameters and variables
T = 1000
beta_start = 0.0001
beta_end = 0.02
beta = torch.linspace(beta_start, beta_end, T)
alpha = 1 - beta
alpha = torch.cumprod(alpha, dim=0)
alpha_T = alpha[-1].to(device)  # Move alpha_T to the same device as data and x

model.eval()

total_val_loss = 0  # Initialize total validation loss

with torch.no_grad():
    for batch_idx, (data, _) in enumerate(val_loader):
        data = data.to(device)  # Reassign to ensure data is on the device
        x = torch.randn(data.shape[0], 1, 28, 28).to(device)
        x = torch.sqrt(alpha_T) * data + torch.sqrt(1 - alpha_T) * x
        
        for s in range(T-1, 0, -1):
            t = torch.tensor([s] * x.size(0), dtype=torch.long).to(device)
            x_hat = model.decoder(x, t)
            z_hat = (1.0 / torch.sqrt(1 - alpha[s])) * (x - torch.sqrt(alpha[s]) * x_hat)
            D_s_minus_one = torch.sqrt(alpha[s-1]) * x_hat + torch.sqrt(1 - alpha[s-1]) * z_hat
            D_s = torch.sqrt(alpha[s]) * x_hat + torch.sqrt(1 - alpha[s]) * z_hat
            x = x - D_s + D_s_minus_one
        
        # Calculate the validation reconstruction loss
        reconstruction_loss = torch.nn.functional.mse_loss(data, x)
        print(f"Reconstruction loss for the batch: {reconstruction_loss.item()}")
        total_val_loss += reconstruction_loss.item()

avg_val_loss = total_val_loss / len(val_loader)

print(f"Average reconstruction loss: {avg_val_loss}")


Reconstruction loss for the batch: 0.22865349054336548
Reconstruction loss for the batch: 0.23499831557273865
Reconstruction loss for the batch: 0.21832510828971863
Reconstruction loss for the batch: 0.2335416078567505
Reconstruction loss for the batch: 0.2325190305709839
Reconstruction loss for the batch: 0.22289256751537323
Reconstruction loss for the batch: 0.21880900859832764
Reconstruction loss for the batch: 0.22996217012405396
Reconstruction loss for the batch: 0.2246590554714203
Reconstruction loss for the batch: 0.21799935400485992
Reconstruction loss for the batch: 0.24871467053890228
Reconstruction loss for the batch: 0.2458362579345703
Reconstruction loss for the batch: 0.2391556352376938
Reconstruction loss for the batch: 0.23485922813415527
Reconstruction loss for the batch: 0.2227887213230133
Reconstruction loss for the batch: 0.22802983224391937
Reconstruction loss for the batch: 0.227611243724823
Reconstruction loss for the batch: 0.23441947996616364
Reconstruction los

In [ ]:

# Save the dictionary to a .pt file
torch.save(sampled_data, 'mnist_cold_alg2.pt')

print("Sampled data saved to 'mnist_cold_alg2.pt'")

In [ ]:
import torch
import matplotlib.pyplot as plt

# Load the saved data
sampled_data = torch.load('mnist_cold_alg2.pt')

# Access a specific entry
index = 50 # Replace with the index you want to check
sampled_image = sampled_data[index]['sampled']


# Convert the tensor to a numpy array
sampled_image = sampled_image.cpu().numpy().transpose(1, 2, 0)


# Display the sampled image
plt.figure(figsize=(5, 5))
plt.imshow(sampled_image, cmap = 'gray')
plt.title("Sampled Image")
plt.axis('off')
plt.show()
